## Building a Recommendation engine from scratch using Julia


### What is Julia : 

Julia is a **high-level, high-performance dynamic programming language for technical computing**, with syntax that is familiar to users of other technical computing environments. It provides a sophisticated compiler, distributed parallel execution, numerical accuracy, and an extensive mathematical function library.

To summarize the movtivation behind creating yet another language here is a quote from the creators of the Julia language, 

> We want a language that’s open source, with a liberal license. We want the speed of C with the dynamism of Ruby. We want a language that’s homoiconic, with true macros like Lisp, but with obvious, familiar mathematical notation like Matlab. We want something as usable for general programming as Python, as easy for statistics as R, as natural for string processing as Perl, as powerful for linear algebra as Matlab, as good at gluing programs together as the shell. Something that is dirt simple to learn, yet keeps the most serious hackers happy. We want it interactive and we want it compiled.

> (Did we mention it should be as fast as C?)

> While we’re being demanding, we want something that provides the distributed power of Hadoop — without the kilobytes of boilerplate Java and XML; without being forced to sift through gigabytes of log files on hundreds of machines to find our bugs. We want the power without the layers of impenetrable complexity. We want to write simple scalar loops that compile down to tight machine code using just the registers on a single CPU. We want to write A*B and launch a thousand computations on a thousand machines, calculating a vast matrix product together.



### What is a Recommendation System : 

* A system which understands the users personal taste for various products.
* The products can range from items on e-commerce sites to movies to connections on social sites.
* It uses the users past behaviour when available, like previous ratings on movies watched previously etc.
* Mathematically models the user-item interaction using complex algorithms.
* Works on large sparse datasets, which consists of the past behaviour.
* Make high quality predictions to individual users.

P.S : Certain limitations do exist, like lack of sufficient information of the users and movies. We assume that each user must rate a certain minimum number of movies in order to get accurate predictions.


### What it takes to build one :

 1. ** A versatile programming language **.
 2. ** Powerful mathematical capabilities **.
 3. High-performance, parallelism, ability to scale.
 4. Web, javascript capabilities.
 5. Fast processing of real-time data.
 
### Why Julia :

1. Its syntax which is similar Python and Matlab. 
2. Julia runs inherently fast, written well.
3. Easy to call C/Fortran.
4. Ready for handling Big Data.
5. Add more.. 

### Toy Model of the movie recommender system :

Consider 4 users *Alice*, *Bob*, *John*, *Dow* and 4 movies *Titanic*, *Braveheart*, *Lion King*, *Troll 2*.

|   |Titanic   |Braveheart   |Lion King   | Troll 2  |
|---|:---:|:---:|:---:|:---:|
|Alice| 4  | 5  | 1  | ?  |
| Bob  | 5  | ?  | 2  | 1  |
| John  | 2  | ?  | 4  | 4  |
| Dow  | ?  | 2  | 2  | 4  |

The table above shows the ratings given by the 4 users to the 4 movies. The `?` are the movies unseen and hence we need to predict the ratings. By setting a threshold, say 3.5, we can recommend movies whose predictions are greater than 3.5.

In [1]:
# Rating martix, R

R = [4 5 1 0;
     5 0 2 1;
     2 0 4 4;
     0 2 2 4;]

4×4 Array{Int64,2}:
 4  5  1  0
 5  0  2  1
 2  0  4  4
 0  2  2  4

In [2]:
typeof(R)
# To add another user, Gary, 
# Initialise an empty array
Gary = Int[]
push!(Gary, 1)
push!(Gary, 1)
push!(Gary, 0)
push!(Gary, 5)

# Now include this into R
R = [R; Gary']

5×4 Array{Int64,2}:
 4  5  1  0
 5  0  2  1
 2  0  4  4
 0  2  2  4
 1  1  0  5

#### Low rank matrix approximations to predict the movie ratings :

In [3]:
# First let us filter out the unwatched movie ratings.
R_s = sparse(R)

5×4 SparseMatrixCSC{Int64,Int64} with 15 stored entries:
  [1, 1]  =  4
  [2, 1]  =  5
  [3, 1]  =  2
  [5, 1]  =  1
  [1, 2]  =  5
  [4, 2]  =  2
  [5, 2]  =  1
  [1, 3]  =  1
  [2, 3]  =  2
  [3, 3]  =  4
  [4, 3]  =  2
  [2, 4]  =  1
  [3, 4]  =  4
  [4, 4]  =  4
  [5, 4]  =  5

In [4]:
# Let `k=3` be the reduced rank
k=3
LR = svds(R_s, nsv=k)

(Base.LinAlg.SVD{Float64,Float64,Array{Float64,2}}([-0.436142 0.718382 0.476358; -0.424157 0.352995 -0.627172; … ; -0.413084 -0.336647 0.362626; -0.409859 -0.39429 0.300228], [9.90289, 6.10092, 4.22203], [-0.540707 -0.345024 -0.431119 -0.634611; 0.597035 0.413762 -0.0741572 -0.683266; -0.408676 0.807021 -0.389202 0.173846]), 6, 1, 9, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])

In [5]:
# Let us reconstruct back the matrix
U = LR[1][:U]
S = LR[1][:S]
V = LR[1][:Vt]

3×4 Array{Float64,2}:
 -0.540707  -0.345024  -0.431119   -0.634611
  0.597035   0.413762  -0.0741572  -0.683266
 -0.408676   0.807021  -0.389202    0.173846

In [6]:
diagm(S)

3×3 Array{Float64,2}:
 9.90289  0.0      0.0    
 0.0      6.10092  0.0    
 0.0      0.0      4.22203

In [8]:
R_new = convert(Array{Int64}, ceil.(U*(diagm(S)*V)))

5×4 Array{Int64,2}:
 5  5  1  1
 5  1  3  1
 3  0  4  5
 1  2  2  5
 1  2  2  5

In [14]:
norm(R.-R_new)

2.6180339887498945

#### Make predictions :

From the above matrix, the recommendations can be made on the predicted unknown ratings.

### Ax = b

In [16]:
 A = [1 2 3; 4 1 6; 7 8 1]

3×3 Array{Int64,2}:
 1  2  3
 4  1  6
 7  8  1

In [17]:
inv(A)

3×3 Array{Float64,2}:
 -0.451923   0.211538    0.0865385
  0.365385  -0.192308    0.0576923
  0.240385   0.0576923  -0.0673077

In [20]:
b = [1 2 3]'

3×1 Array{Int64,2}:
 1
 2
 3

In [21]:
#b \ A

1×3 Array{Float64,2}:
 2.14286  2.0  1.28571

In [32]:
using Images, ImageMagick
#using TSVD
using Interact

x = load("./images/strang.jpg")
img = map(x->Float64(x.r), x)
@manipulate for i=1:100
    LR = svds(img, nsv=i)
    U = LR[1][:U]
    S = LR[1][:S]
    V = LR[1][:Vt]
    map(U * (diagm(S) * V)) do a
        RGB(a, a, a)
    end
end

LoadError: [91mUndefVarError: load not defined[39m

In [31]:
;ls images

als.png
fizz.jpg
strang.jpg


### Exploratory analysis and plots of the data :

### Parallel Movie recommender system 

This notebook demos collaborative filtering based movie recommender systems in Julia. The package [RecSys.jl](https://github.com/abhijithch/RecSys.jl/) is a package for recommender systems in Julia, it can currently work with explicit ratings data. This demos a parallel implementation of the ALS factorization based collaborative filtering for movie recommendations based on [this](http://dl.acm.org/citation.cfm?id=1424269) research article. The detailed report of the system is [here](http://juliacomputing.com/blog/2016/04/22/a-parallel-recommendation-engine-in-julia.html).

### Collaborative Filtering using weighted ALS factorization :

<img src="./images/als.png" width="550">

Let $U={u_i}$ be the user feature matrix where ${u_i} \subseteq\mathbb{R}^{n_f}$ and $i=1,2,...,n_u$, and let $M={m_j}$ be the item or movie feature matrix, where ${m_j} \subseteq \mathbb{R}^{n_f}$ and $j=1,2,...,n_m$. Here $n_f$ is the number of factors, i.e., the reduced dimension or the lower rank, which is determined by cross validation. The predictions can be calculated for any user-movie combination,
$(i,j)$, as $r_{ij}={u_i} \cdotp {m_j}, \forall i,j$.

** Credits ** :

[Large-Scale Parallel Collaborative Filtering for the Netflix Prize](http://dl.acm.org/citation.cfm?id=1424269)

[Movielens dataset](http://grouplens.org/datasets/movielens/)

In [2]:
# Installation of the packages. (To be done the first time, hence comment after installing.)
Pkg.clone("https://github.com/tanmaykm/Blobs.jl.git")
Pkg.clone("https://github.com/abhijithch/RecSys.jl")

INFO: Cloning Blobs from https://github.com/tanmaykm/Blobs.jl.git
INFO: Computing changes...
INFO: No packages to install, update or remove
INFO: Cloning RecSys from https://github.com/abhijithch/RecSys.jl
INFO: Computing changes...
INFO: Cloning cache of ArrayViews from https://github.com/JuliaArrays/ArrayViews.jl.git
INFO: Cloning cache of MAT from https://github.com/JuliaIO/MAT.jl.git
INFO: Cloning cache of ParallelSparseMatMul from https://github.com/madeleineudell/ParallelSparseMatMul.jl.git
INFO: Cloning cache of SparseVectors from https://github.com/JuliaSparse/SparseVectors.jl.git
INFO: Installing ArrayViews v0.6.4
INFO: Installing MAT v0.4.0
INFO: Installing ParallelSparseMatMul v0.0.1
INFO: Installing SparseVectors v0.4.2
INFO: Building Homebrew


Already up-to-date.


INFO: Building Blosc
INFO: Building HDF5


In [3]:
workspace()
using RecSys
include(joinpath(Pkg.dir("RecSys"), "examples", "movielens", "movielens.jl"))


Use "const RatingMatrix = SparseMatrixCSC{Float64,Int64}" instead.

Use "const SharedRatingMatrix = ParallelSparseMatMul.SharedSparseMatrixCSC{Float64,Int64}" instead.

Use "const InputRatings = Union{RatingMatrix,SharedRatingMatrix,SparseMatBlobs}" instead.

Use "const InputIdMap = Union{Vector{Int64},SharedVector{Int64}}" instead.

Use "const ModelFactor = Union{Matrix{Float64},SharedArray{Float64,2},DenseMatBlobs{Float64}}" instead.

Use "abstract type FileSpec end" instead.

Use "abstract type Inputs end" instead.

Use "abstract type Model end" instead.

Use "abstract type Parallelism end" instead.
INFO: Precompiling module MAT.


INFO: Precompiling module Blobs.

Use "const MatBlobs = Union{SparseMatBlobs,DenseMatBlobs}" instead.


test_chunks (generic function with 1 method)

### Dataset : 

GroupLens Research has collected and made available rating data sets from the [MovieLens](http://movielens.org) web site. The data sets were collected over various periods of time, depending on the size of the set. 

#### MovieLens 20M Dataset

Stable benchmark dataset. 20 million ratings and 465,000 tag applications applied to 27,000 movies by 138,000 users. Includes tag genome data with 12 million relevance scores across 1,100 tags.

We use the ratings data to form a sparse matrix of size `138,000 X 27,000` with 20 million ratings ranging from 1 to 5.

In [4]:
# Please specify path to the data folder which includes the 20 million ratings data folder "ml-20m"
dataset_path = "/Users/abhijith/work/ML/notebooks/data/recommender"

"/Users/abhijith/work/ML/notebooks/data/recommender"

In [5]:
data_folder = "ml-20m"

"ml-20m"

Creating file handles to the movie ratings and the movies list files.

In [6]:
ratings_file = DlmFile(joinpath(dataset_path,data_folder, "ratings.csv"); dlm=',', header=true)
movies_file = DlmFile(joinpath(dataset_path,data_folder, "movies.csv"); dlm=',', header=true)

RecSys.DlmFile("/Users/abhijith/work/ML/notebooks/data/recommender/ml-20m/movies.csv", ',', true, true)

#### Parallel implementations :

This package offers 3 modes of parallelism, 

1. Multi-threading - Julia native threading infrastructure provides an easy way to make use threads.
2. Shared memory - This is a multiprocessing using shared data.
3. Distributed memory - This is distributed memory based multiprocessing, this would require that the data be split into chunks. There is code to do this, refer ...

Multiple Dispatch is a nice feature in Julia, which would dispatch to the correct implementation based on the type of the objects passed as arguments. 

For e.x., if we need to train the model using shared memory multiprocessing, the type of `MovieRec` is `MovieRec(trainingset::FileSpec, movie_names::FileSpec)` and if we need distributed memory model the type of `MovieRec` is `MovieRec(user_item_ratings::FileSpec, item_user_ratings::FileSpec, movie_names::FileSpec)`.

Let us see how Shared memory Parallel implementation trains the MovieLens data set

In [7]:
rec = MovieRec(ratings_file, movies_file)

MovieRec(RecSys.DlmFile("/Users/abhijith/work/ML/notebooks/data/recommender/ml-20m/movies.csv", ',', true, true), RecSys.ALSWR{RecSys.ParShmem,RecSys.SharedMemoryInputs,RecSys.SharedMemoryModel}(RecSys.SharedMemoryInputs(RecSys.DlmFile("/Users/abhijith/work/ML/notebooks/data/recommender/ml-20m/ratings.csv", ',', true, true), 0, 0, Nullable{Union{ParallelSparseMatMul.SharedSparseMatrixCSC{Float64,Int64}, RecSys.MatrixBlobs.SparseMatBlobs, SparseMatrixCSC{Float64,Int64}}}(), Nullable{Union{ParallelSparseMatMul.SharedSparseMatrixCSC{Float64,Int64}, RecSys.MatrixBlobs.SparseMatBlobs, SparseMatrixCSC{Float64,Int64}}}(), Nullable{Union{Array{Int64,1}, SharedArray{Int64,1}}}(), Nullable{Union{Array{Int64,1}, SharedArray{Int64,1}}}()), Nullable{RecSys.SharedMemoryModel}(), RecSys.ParShmem()), Nullable{SparseVector{AbstractString,Int64}}())

In [8]:
movies_file.name

"/Users/abhijith/work/ML/notebooks/data/recommender/ml-20m/movies.csv"

To run in parallel mode, add processes like below:

Let us train the model with `10` factors and `10` iterations.

In [9]:
@time train(rec, 10, 10)

Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:70
 [2] #SharedArray#773(::Array{Any,1}, ::Type{T} where T, ::Type{Float64}, ::Tuple{Int64,Int64}) at ./deprecated.jl:57
 [3] SharedArray(::Type{Float64}, ::Tuple{Int64,Int64}) at ./deprecated.jl:57
 [4] #share#2(::Array{Any,1}, ::Function, ::Array{Float64,2}) at /Applications/JuliaPro-0.6.2.2.app/Contents/Resources/pkgs-0.6.2.2/v0.6/ParallelSparseMatMul/src/parallel_matmul.jl:38
 [5] share(::Array{Float64,2}) at /Applications/JuliaPro-0.6.2.2.app/Contents/Resources/pkgs-0.6.2.2/v0.6/ParallelSparseMatMul/src/parallel_matmul.jl:38
 [6] share!(::RecSys.SharedMemoryModel) at /Applications/JuliaPro-0.6.2.2.app/Contents/Resources/pkgs-0.6.2.2/v0.6/RecSys/src/models/als_model.jl:15
 [7] fact_iters(::RecSys.ParShmem, ::RecSys.SharedMemoryModel, ::RecSys.SharedMemoryInputs, ::Int64) at /Applications/JuliaPro-0.6.2.2.app/Contents/Resources/pkgs-0.6.2.2/v0.6/RecSys/src/als-wr.jl:192
 [8] fact_iters(::RecSys.ALSWR{RecSys.ParShmem,Rec

 99.095314 seconds (45.81 M allocations: 64.176 GiB, 9.17% gc time)


In [150]:
nprocs()

1

In [10]:
err = rmse(rec)

Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:70
 [2] #SharedArray#773(::Array{Any,1}, ::Type{T} where T, ::Type{Float64}, ::Tuple{Int64,Int64}) at ./deprecated.jl:57
 [3] SharedArray(::Type{Float64}, ::Tuple{Int64,Int64}) at ./deprecated.jl:57
 [4] #share#2(::Array{Any,1}, ::Function, ::Array{Float64,2}) at /Applications/JuliaPro-0.6.2.2.app/Contents/Resources/pkgs-0.6.2.2/v0.6/ParallelSparseMatMul/src/parallel_matmul.jl:38
 [5] share(::Array{Float64,2}) at /Applications/JuliaPro-0.6.2.2.app/Contents/Resources/pkgs-0.6.2.2/v0.6/ParallelSparseMatMul/src/parallel_matmul.jl:38
 [6] share!(::RecSys.SharedMemoryModel) at /Applications/JuliaPro-0.6.2.2.app/Contents/Resources/pkgs-0.6.2.2/v0.6/RecSys/src/models/als_model.jl:15
 [7] #rmse#20(::Int64, ::Function, ::RecSys.ALSWR{RecSys.ParShmem,RecSys.SharedMemoryInputs,RecSys.SharedMemoryModel}, ::RecSys.SharedMemoryInputs) at /Applications/JuliaPro-0.6.2.2.app/Contents/Resources/pkgs-0.6.2.2/v0.6/RecSys/src/als-wr.jl:247
 [8

Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:70
 [2] (::Base.##715#716)(::Array{Float64,1}, ::Array{Float64,1}) at ./deprecated.jl:355
 [3] (::RecSys.##21#22{RecSys.SharedMemoryInputs,RecSys.SharedMemoryModel})(::Base.##715#716, ::Array{UnitRange{Int64},1}, ::Int64, ::Int64) at ./distributed/macros.jl:162
 [4] (::Base.Distributed.##135#136{RecSys.##21#22{RecSys.SharedMemoryInputs,RecSys.SharedMemoryModel},Tuple{Base.##715#716,Array{UnitRange{Int64},1},Int64,Int64},Array{Any,1}})() at ./distributed/remotecall.jl:314
 [5] run_work_thunk(::Base.Distributed.##135#136{RecSys.##21#22{RecSys.SharedMemoryInputs,RecSys.SharedMemoryModel},Tuple{Base.##715#716,Array{UnitRange{Int64},1},Int64,Int64},Array{Any,1}}, ::Bool) at ./distributed/process_messages.jl:56
 [6] #remotecall_fetch#140(::Array{Any,1}, ::Function, ::Function, ::Base.Distributed.LocalProcess, ::Function, ::Vararg{Any,N} where N) at ./distributed/remotecall.jl:339
 [7] remotecall_fetch(::Function, ::Base.Distribu

Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:70
 [2] (::Base.##715#716)(::Array{Float64,1}, ::Array{Float64,1}) at ./deprecated.jl:355
 [3] (::RecSys.##21#22{RecSys.SharedMemoryInputs,RecSys.SharedMemoryModel})(::Base.##715#716, ::Array{UnitRange{Int64},1}, ::Int64, ::Int64) at ./distributed/macros.jl:162
 [4] (::Base.Distributed.##135#136{RecSys.##21#22{RecSys.SharedMemoryInputs,RecSys.SharedMemoryModel},Tuple{Base.##715#716,Array{UnitRange{Int64},1},Int64,Int64},Array{Any,1}})() at ./distributed/remotecall.jl:314
 [5] run_work_thunk(::Base.Distributed.##135#136{RecSys.##21#22{RecSys.SharedMemoryInputs,RecSys.SharedMemoryModel},Tuple{Base.##715#716,Array{UnitRange{Int64},1},Int64,Int64},Array{Any,1}}, ::Bool) at ./distributed/process_messages.jl:56
 [6] #remotecall_fetch#140(::Array{Any,1}, ::Function, ::Function, ::Base.Distributed.LocalProcess, ::Function, ::Vararg{Any,N} where N) at ./distributed/remotecall.jl:339
 [7] remotecall_fetch(::Function, ::Base.Distribu

Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:70
 [2] (::Base.##715#716)(::Array{Float64,1}, ::Array{Float64,1}) at ./deprecated.jl:355
 [3] (::RecSys.##21#22{RecSys.SharedMemoryInputs,RecSys.SharedMemoryModel})(::Base.##715#716, ::Array{UnitRange{Int64},1}, ::Int64, ::Int64) at ./distributed/macros.jl:162
 [4] (::Base.Distributed.##135#136{RecSys.##21#22{RecSys.SharedMemoryInputs,RecSys.SharedMemoryModel},Tuple{Base.##715#716,Array{UnitRange{Int64},1},Int64,Int64},Array{Any,1}})() at ./distributed/remotecall.jl:314
 [5] run_work_thunk(::Base.Distributed.##135#136{RecSys.##21#22{RecSys.SharedMemoryInputs,RecSys.SharedMemoryModel},Tuple{Base.##715#716,Array{UnitRange{Int64},1},Int64,Int64},Array{Any,1}}, ::Bool) at ./distributed/process_messages.jl:56
 [6] #remotecall_fetch#140(::Array{Any,1}, ::Function, ::Function, ::Base.Distributed.LocalProcess, ::Function, ::Vararg{Any,N} where N) at ./distributed/remotecall.jl:339
 [7] remotecall_fetch(::Function, ::Base.Distribu

Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:70
 [2] (::Base.##715#716)(::Array{Float64,1}, ::Array{Float64,1}) at ./deprecated.jl:355
 [3] (::RecSys.##21#22{RecSys.SharedMemoryInputs,RecSys.SharedMemoryModel})(::Base.##715#716, ::Array{UnitRange{Int64},1}, ::Int64, ::Int64) at ./distributed/macros.jl:162
 [4] (::Base.Distributed.##135#136{RecSys.##21#22{RecSys.SharedMemoryInputs,RecSys.SharedMemoryModel},Tuple{Base.##715#716,Array{UnitRange{Int64},1},Int64,Int64},Array{Any,1}})() at ./distributed/remotecall.jl:314
 [5] run_work_thunk(::Base.Distributed.##135#136{RecSys.##21#22{RecSys.SharedMemoryInputs,RecSys.SharedMemoryModel},Tuple{Base.##715#716,Array{UnitRange{Int64},1},Int64,Int64},Array{Any,1}}, ::Bool) at ./distributed/process_messages.jl:56
 [6] #remotecall_fetch#140(::Array{Any,1}, ::Function, ::Function, ::Base.Distributed.LocalProcess, ::Function, ::Vararg{Any,N} where N) at ./distributed/remotecall.jl:339
 [7] remotecall_fetch(::Function, ::Base.Distribu

...

Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:70
 [2] (::Base.##715#716)(::Array{Float64,1}, ::Array{Float64,1}) at ./deprecated.jl:355
 [3] (::RecSys.##21#22{RecSys.SharedMemoryInputs,RecSys.SharedMemoryModel})(::Base.##715#716, ::Array{UnitRange{Int64},1}, ::Int64, ::Int64) at ./distributed/macros.jl:162
 [4] (::Base.Distributed.##135#136{RecSys.##21#22{RecSys.SharedMemoryInputs,RecSys.SharedMemoryModel},Tuple{Base.##715#716,Array{UnitRange{Int64},1},Int64,Int64},Array{Any,1}})() at ./distributed/remotecall.jl:314
 [5] run_work_thunk(::Base.Distributed.##135#136{RecSys.##21#22{RecSys.SharedMemoryInputs,RecSys.SharedMemoryModel},Tuple{Base.##715#716,Array{UnitRange{Int64},1},Int64,Int64},Array{Any,1}}, ::Bool) at ./distributed/process_messages.jl:56
 [6] #remotecall_fetch#140(::Array{Any,1}, ::Function, ::Function, ::Base.Distributed.LocalProcess, ::Function, ::Vararg{Any,N} where N) at ./distributed/remotecall.jl:339
 [7] remotecall_fetch(::Function, ::Base.Distribu

Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:70
 [2] (::Base.##715#716)(::Array{Float64,1}, ::Array{Float64,1}) at ./deprecated.jl:355
 [3] (::RecSys.##21#22{RecSys.SharedMemoryInputs,RecSys.SharedMemoryModel})(::Base.##715#716, ::Array{UnitRange{Int64},1}, ::Int64, ::Int64) at ./distributed/macros.jl:162
 [4] (::Base.Distributed.##135#136{RecSys.##21#22{RecSys.SharedMemoryInputs,RecSys.SharedMemoryModel},Tuple{Base.##715#716,Array{UnitRange{Int64},1},Int64,Int64},Array{Any,1}})() at ./distributed/remotecall.jl:314
 [5] run_work_thunk(::Base.Distributed.##135#136{RecSys.##21#22{RecSys.SharedMemoryInputs,RecSys.SharedMemoryModel},Tuple{Base.##715#716,Array{UnitRange{Int64},1},Int64,Int64},Array{Any,1}}, ::Bool) at ./distributed/process_messages.jl:56
 [6] #remotecall_fetch#140(::Array{Any,1}, ::Function, ::Function, ::Base.Distributed.LocalProcess, ::Function, ::Vararg{Any,N} where N) at ./distributed/remotecall.jl:339
 [7] remotecall_fetch(::Function, ::Base.Distribu

Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:70
 [2] (::Base.##715#716)(::Array{Float64,1}, ::Array{Float64,1}) at ./deprecated.jl:355
 [3] (::RecSys.##21#22{RecSys.SharedMemoryInputs,RecSys.SharedMemoryModel})(::Base.##715#716, ::Array{UnitRange{Int64},1}, ::Int64, ::Int64) at ./distributed/macros.jl:162
 [4] (::Base.Distributed.##135#136{RecSys.##21#22{RecSys.SharedMemoryInputs,RecSys.SharedMemoryModel},Tuple{Base.##715#716,Array{UnitRange{Int64},1},Int64,Int64},Array{Any,1}})() at ./distributed/remotecall.jl:314
 [5] run_work_thunk(::Base.Distributed.##135#136{RecSys.##21#22{RecSys.SharedMemoryInputs,RecSys.SharedMemoryModel},Tuple{Base.##715#716,Array{UnitRange{Int64},1},Int64,Int64},Array{Any,1}}, ::Bool) at ./distributed/process_messages.jl:56
 [6] #remotecall_fetch#140(::Array{Any,1}, ::Function, ::Function, ::Base.Distributed.LocalProcess, ::Function, ::Vararg{Any,N} where N) at ./distributed/remotecall.jl:339
 [7] remotecall_fetch(::Function, ::Base.Distribu

Excessive output truncated after 526900 bytes.WARNING: .+ is no longer a function object; use broadcast(+, ...) instead
Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:70
 [2] (::Base.##715#716)(::Array{Float64,1}, ::Array{Float64,1}) at ./deprecated.jl:355
 [3] (::RecSys.##21#22{RecSys.SharedMemoryInputs,RecSys.SharedMemoryModel})(::Base.##715#716, ::Array{UnitRange{Int64},1}, ::Int64, ::Int64) at ./distributed/macros.jl:162
 [4] (::Base.Distributed.##135#136{RecSys.##21#22{RecSys.SharedMemoryInputs,RecSys.SharedMemoryModel},Tuple{Base.##715#716,Array{UnitRange{Int64},1},Int64,Int64},Array{Any,1}})() at ./distributed/remotecall.jl:314
 [5] run_work_thunk(::Base.Distributed.##135#136{RecSys.##21#22{RecSys.SharedMemoryInputs,RecSys.SharedMemoryModel},Tuple{Base.##715#716,Array{UnitRange{Int64},1},Int64,Int64},Array{Any,1}}, ::Bool) at ./distributed/process_messages.jl:56
 [6] #remotecall_fetch#140(::Array{Any,1}, ::Function, ::Function, ::Base.Distributed.LocalProcess, :

0.7535891594760539

#### Parallel Run:

In [ ]:
# Check the number of procs
nprocs()
# Add any number of procs
#addprocs(2)
@everywhere using RecSys
@everywhere include(joinpath(Pkg.dir("RecSys"), "examples", "movielens", "movielens.jl"))

In [ ]:
@time train(rec, 10, 10)

#### Select a user, for which we show the movies watched and the recommendations for the user. 

In [38]:
user = 30
print_recommendations(rec, recommend(rec, user)...)

Already watched:
  [1 ]  =  "Father of the Bride Part II (1995) - Comedy"
  [2 ]  =  "Desperado (1995) - Action|Romance|Western"
  [3 ]  =  "First Knight (1995) - Action|Drama|Romance"
  [4 ]  =  "Disclosure (1994) - Drama|Thriller"
  [5 ]  =  "Like Water for Chocolate (Como agua para chocolate) (1992) - Drama|Fantasy|Romance"
  [6 ]  =  "Hot Shots! Part Deux (1993) - Action|Comedy|War"
  [7 ]  =  "RoboCop 3 (1993) - Action|Crime|Drama|Sci-Fi|Thriller"
  [8 ]  =  "True Romance (1993) - Crime|Thriller"
  [9 ]  =  "Terminator 2: Judgment Day (1991) - Action|Sci-Fi"
  [10]  =  "Pinocchio (1940) - Animation|Children|Fantasy|Musical"
  [11]  =  "Sleepers (1996) - Thriller"
  [12]  =  "Army of Darkness (1993) - Action|Adventure|Comedy|Fantasy|Horror"
  [13]  =  "Sting, The (1973) - Comedy|Crime"
  [14]  =  "Terminator, The (1984) - Action|Sci-Fi|Thriller"
  [15]  =  "Star Trek VI: The Undiscovered Country (1991) - Action|Mystery|Sci-Fi"
  [16]  =  "Star Trek IV: The Voyage Home (1986) - Adve

In [149]:
M = read

LoadError: UndefVarError: M not defined